# 할인율과 매출액 상관관계 분석

이 노트북은 discount_sales_data.csv 파일의 데이터를 분석하여 할인율과 매출 간의 상관관계를 시각화합니다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [ ]:
# 데이터 로드
df = pd.read_csv(r'c:\Users\MADUP\Downloads\discount_sales_data.csv')

# 데이터 기본 정보 출력
print("="*60)
print("데이터 기본 정보")
print("="*60)
print(df.head(10))
print(f"\n전체 데이터 수: {len(df)}개")
print(f"\n데이터 타입:\n{df.dtypes}")

In [ ]:
# 기초 통계량
print("="*60)
print("기초 통계량")
print("="*60)
df[['할인율', '매출액', '판매수량']].describe()

In [ ]:
# 상관관계 분석
print("="*60)
print("상관관계 분석")
print("="*60)

# 피어슨 상관계수
correlation = df['할인율'].corr(df['매출액'])
print(f"\n할인율과 매출액의 피어슨 상관계수: {correlation:.4f}")

# 스피어만 상관계수 (순위 기반)
spearman_corr, spearman_p = stats.spearmanr(df['할인율'], df['매출액'])
print(f"할인율과 매출액의 스피어만 상관계수: {spearman_corr:.4f} (p-value: {spearman_p:.4f})")

In [ ]:
# 할인율별 평균 매출 분석
print("="*60)
print("할인율별 평균 매출 및 판매 현황")
print("="*60)
discount_analysis = df.groupby('할인율').agg({
    '매출액': ['mean', 'sum', 'count'],
    '판매수량': 'mean'
}).round(2)
discount_analysis

In [ ]:
# 시각화
fig = plt.figure(figsize=(16, 12))

# 1. 산점도와 추세선
ax1 = plt.subplot(2, 3, 1)
plt.scatter(df['할인율'], df['매출액'], alpha=0.6, s=100, c=df['할인율'], cmap='viridis')
z = np.polyfit(df['할인율'], df['매출액'], 1)
p = np.poly1d(z)
plt.plot(df['할인율'], p(df['할인율']), "r--", linewidth=2, label=f'추세선 (상관계수: {correlation:.3f})')
plt.xlabel('할인율 (%)', fontsize=12, fontweight='bold')
plt.ylabel('매출액 (원)', fontsize=12, fontweight='bold')
plt.title('할인율 vs 매출액 산점도', fontsize=14, fontweight='bold', pad=15)
plt.legend()
plt.grid(True, alpha=0.3)
plt.colorbar(label='할인율 (%)')

# 2. 할인율별 평균 매출 박스플롯
ax2 = plt.subplot(2, 3, 2)
df.boxplot(column='매출액', by='할인율', ax=ax2)
plt.xlabel('할인율 (%)', fontsize=12, fontweight='bold')
plt.ylabel('매출액 (원)', fontsize=12, fontweight='bold')
plt.title('할인율별 매출액 분포', fontsize=14, fontweight='bold', pad=15)
plt.suptitle('')  # 기본 제목 제거

# 3. 할인율별 평균 매출 막대 그래프
ax3 = plt.subplot(2, 3, 3)
avg_sales = df.groupby('할인율')['매출액'].mean()
bars = plt.bar(avg_sales.index, avg_sales.values, color=plt.cm.viridis(np.linspace(0, 1, len(avg_sales))), 
               edgecolor='black', linewidth=1.5)
plt.xlabel('할인율 (%)', fontsize=12, fontweight='bold')
plt.ylabel('평균 매출액 (원)', fontsize=12, fontweight='bold')
plt.title('할인율별 평균 매출액', fontsize=14, fontweight='bold', pad=15)
plt.grid(True, alpha=0.3, axis='y')
# 값 표시
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}원',
             ha='center', va='bottom', fontsize=9, fontweight='bold')

# 4. 할인율별 총 매출 파이 차트
ax4 = plt.subplot(2, 3, 4)
total_sales = df.groupby('할인율')['매출액'].sum()
colors = plt.cm.viridis(np.linspace(0, 1, len(total_sales)))
wedges, texts, autotexts = plt.pie(total_sales.values, labels=[f'{x}%' for x in total_sales.index], 
                                     autopct='%1.1f%%', colors=colors, startangle=90,
                                     textprops={'fontsize': 10, 'fontweight': 'bold'})
plt.title('할인율별 총 매출 비중', fontsize=14, fontweight='bold', pad=15)

# 5. 할인율별 상품 수
ax5 = plt.subplot(2, 3, 5)
product_count = df.groupby('할인율').size()
bars = plt.bar(product_count.index, product_count.values, color=plt.cm.plasma(np.linspace(0, 1, len(product_count))),
               edgecolor='black', linewidth=1.5)
plt.xlabel('할인율 (%)', fontsize=12, fontweight='bold')
plt.ylabel('상품 수', fontsize=12, fontweight='bold')
plt.title('할인율별 상품 수', fontsize=14, fontweight='bold', pad=15)
plt.grid(True, alpha=0.3, axis='y')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}개',
             ha='center', va='bottom', fontsize=9, fontweight='bold')

# 6. 할인율별 평균 판매수량
ax6 = plt.subplot(2, 3, 6)
avg_quantity = df.groupby('할인율')['판매수량'].mean()
bars = plt.bar(avg_quantity.index, avg_quantity.values, color=plt.cm.coolwarm(np.linspace(0, 1, len(avg_quantity))),
               edgecolor='black', linewidth=1.5)
plt.xlabel('할인율 (%)', fontsize=12, fontweight='bold')
plt.ylabel('평균 판매수량', fontsize=12, fontweight='bold')
plt.title('할인율별 평균 판매수량', fontsize=14, fontweight='bold', pad=15)
plt.grid(True, alpha=0.3, axis='y')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}개',
             ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('discount_sales_analysis.png', dpi=300, bbox_inches='tight')
print("\n시각화 결과가 'discount_sales_analysis.png'로 저장되었습니다.")
plt.show()

In [ ]:
# 추가 인사이트 분석
print("="*60)
print("분석 인사이트")
print("="*60)

if abs(correlation) < 0.3:
    corr_strength = "약한"
elif abs(correlation) < 0.7:
    corr_strength = "중간 정도의"
else:
    corr_strength = "강한"

if correlation > 0:
    corr_direction = "양의"
    interpretation = "할인율이 높을수록 매출이 증가하는 경향"
else:
    corr_direction = "음의"
    interpretation = "할인율이 높을수록 매출이 감소하는 경향"

print(f"\n1. 상관관계 해석:")
print(f"   - {corr_strength} {corr_direction} 상관관계가 있습니다. ({interpretation})")

print(f"\n2. 할인율별 성과:")
best_discount = avg_sales.idxmax()
best_sales = avg_sales.max()
worst_discount = avg_sales.idxmin()
worst_sales = avg_sales.min()
print(f"   - 평균 매출이 가장 높은 할인율: {best_discount}% (평균 {int(best_sales):,}원)")
print(f"   - 평균 매출이 가장 낮은 할인율: {worst_discount}% (평균 {int(worst_sales):,}원)")

print(f"\n3. 판매수량과의 관계:")
quantity_corr = df['할인율'].corr(df['판매수량'])
print(f"   - 할인율과 판매수량의 상관계수: {quantity_corr:.4f}")
print(f"   - 평균 판매수량이 가장 높은 할인율: {avg_quantity.idxmax()}% (평균 {int(avg_quantity.max())}개)")

print(f"\n4. 카테고리별 분석:")
category_analysis = df.groupby('카테고리').agg({
    '할인율': 'mean',
    '매출액': 'mean'
}).round(2)
category_analysis

In [ ]:
# 추가 시각화: 카테고리별 분석
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 카테고리별 평균 매출
category_sales = df.groupby('카테고리')['매출액'].mean().sort_values(ascending=False)
axes[0].barh(category_sales.index, category_sales.values, color='skyblue', edgecolor='black')
axes[0].set_xlabel('평균 매출액 (원)', fontsize=11, fontweight='bold')
axes[0].set_title('카테고리별 평균 매출액', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# 카테고리별 평균 할인율
category_discount = df.groupby('카테고리')['할인율'].mean().sort_values(ascending=False)
axes[1].barh(category_discount.index, category_discount.values, color='lightcoral', edgecolor='black')
axes[1].set_xlabel('평균 할인율 (%)', fontsize=11, fontweight='bold')
axes[1].set_title('카테고리별 평균 할인율', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('category_analysis.png', dpi=300, bbox_inches='tight')
print("카테고리별 분석 결과가 'category_analysis.png'로 저장되었습니다.")
plt.show()